In [1]:
import requests
import numpy as np
import pandas as pd

from io import StringIO

import plotly.express as px
import plotly.graph_objects as go

In [3]:
ENDPOINT = "community-api.coinmetrics.io/v4"

def query(asset: str):
    dfs = []
    next_page_url = f"https://{ENDPOINT}/timeseries/asset-metrics?assets={asset}&metrics=PriceUSD,CapRealUSD,SplyCur&frequency=1d&format=csv&page_size=1000&paging_from=start"
    while next_page_url:
        response = requests.get(next_page_url)
        dfs.append(pd.read_csv(StringIO(response.text)))
        next_page_url = response.headers.get('x-next-page-url', None)
    
    return pd.concat(dfs)

In [12]:
ASSET = "doge"

df = query(ASSET)
df.rename(columns=dict(zip(df.columns, ['asset', 'datetime', 'market_cap', 'price', 'supply'])), inplace=True)

In [35]:
scale_presets = {
    "btc": {
        "ceil_scale": 3.9,
        "floor_scale": 0.78
    },
    "eth": {
        "ceil_scale": 2.4,
        "floor_scale": 0.55
    },
    "ada": {
        "ceil_scale": 2.4,
        "floor_scale": 0.44
    },
    "doge": {
        "ceil_scale": 4.23,
        "floor_scale": 0.32
    },
}

df['realized_price'] = df['market_cap'] / df['supply']
df['ceil_price']     = df['realized_price'] * scale_presets[ASSET]['ceil_scale']
df['floor_price']    = df['realized_price'] * scale_presets[ASSET]['floor_scale']

In [36]:
df = df[df['market_cap'] > 0]

fig = go.Figure()
fig.add_traces([
    # go.Line(x=df['datetime'], y=df['realized_price'], name="realized_price"),
    go.Scatter(x=df['datetime'], y=df['ceil_price'], name="ceil price", mode="lines", line=dict(color="tomato", width=3)),
    go.Scatter(x=df['datetime'], y=df['floor_price'], name="floor price", mode="lines", line=dict(color="limegreen", width=3)),
    go.Scatter(x=df['datetime'], y=df['price'], name="price", mode="lines", line=dict(color="black")),
])

fig.update_layout(title=f"Ceil/Floor Price Prediction of {ASSET.upper()}")
fig.update_yaxes(type="log")
fig.show()

In [37]:
fig.write_html(f"output/{ASSET}.html")